# Online Hardware Validation

Interactive notebook for running **live** hardware validation tests.

**Test Categories:**
- A1: Internal Noise (Shorted Inputs)
- A2: External Noise (Floating Inputs)
- B: Known Signal Injection (KSI)
- C: Functional EEG Tests (Eyes Open/Closed)

**Architecture:**
- `acquisition.capture_live_stream()` - WebSocket frame streaming
- `analysis.run_pipeline()` - Same analysis as offline

---
## Setup & Configuration

Run this cell first to import packages and configure ESP32.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════
import asyncio
import nest_asyncio
nest_asyncio.apply()  # Allow nested asyncio in Jupyter

# Acquisition (live streaming)
from acquisition import HardwareClient, capture_live_stream

# Analysis (same as offline)
from analysis.pipeline import run_pipeline
from analysis.preprocess import FS_HZ

print("✓ Imports successful")
print(f"  Sampling rate: {FS_HZ} Hz")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION - EDIT THESE
# ═══════════════════════════════════════════════════════════════════════════════

ESP32_IP = "node.local"           # ESP32 hostname or IP
CAPTURE_DURATION_S = 5.0          # Capture duration in seconds

# Create hardware client
client = HardwareClient(ESP32_IP)

# Detect sample rate from hardware
sample_rate = client.detect_sample_rate()
if sample_rate:
    print(f"✓ Connected to {ESP32_IP}")
    print(f"  Detected sample rate: {sample_rate} Hz")
else:
    sample_rate = FS_HZ
    print(f"⚠ Could not detect sample rate, using default: {sample_rate} Hz")

---
## Helper: Capture & Analyze

Single function that captures live data and runs the full analysis pipeline.

In [ ]:
def capture_and_analyze(test_name: str, duration_s: float = CAPTURE_DURATION_S):
    """
    Capture live data and run full analysis pipeline.
    
    Args:
        test_name: Name for the test (e.g., "Internal Noise")
        duration_s: Capture duration in seconds
    
    Returns:
        Analysis results dict
    """
    print(f"\n{'='*60}")
    print(f" {test_name} ".center(60, "="))
    print(f"{'='*60}\n")
    
    # Capture live stream
    sample_bytes, raw_frames, info = asyncio.run(
        capture_live_stream(client, duration_s, sample_rate)
    )
    
    if len(raw_frames) == 0:
        print("❌ No frames received - device not ready")
        return None
    
    # Run full analysis pipeline (same as offline!)
    results = run_pipeline(
        raw_frames,              # Pass bytes directly
        source_name=test_name,   # Display name
        display_plots=True,
    )
    
    print(f"\n✓ {test_name} complete.")
    return results

---
## A1: Internal Noise (Shorted Inputs)

Measures the intrinsic noise floor of the ADS1299 ADC with all inputs shorted.

**Setup:** Connect all inputs to BIAS (shorted configuration)

**Expected:** Very low noise, flat spectrum

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# A1: INTERNAL NOISE
# ═══════════════════════════════════════════════════════════════════════════════

results_internal = capture_and_analyze("A1 Internal Noise")

---
## A2: External Noise (Floating Inputs)

Measures environmental noise pickup with floating (unconnected) inputs.

**Setup:** Disconnect all electrode inputs

**Expected:** 50Hz mains pickup, higher noise than internal

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# A2: EXTERNAL NOISE
# ═══════════════════════════════════════════════════════════════════════════════

results_external = capture_and_analyze("A2 External Noise")

---
## B: Known Signal Injection (KSI)

Verifies channel integrity by injecting a known signal.

**Setup:** Connect signal generator to target channel

**Expected:** Clear peak at injected frequency

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# B: KNOWN SIGNAL INJECTION
# ═══════════════════════════════════════════════════════════════════════════════

results_ksi = capture_and_analyze("KSI 40Hz CH2")

---
## C1: Eyes Open

Real EEG recording with eyes open.

**Setup:** Attach EEG electrodes to subject

**Expected:** Baseline brain activity, minimal alpha

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# C1: EYES OPEN
# ═══════════════════════════════════════════════════════════════════════════════

input("Subject: Keep eyes OPEN. Press Enter to start recording...")
results_eo = capture_and_analyze("C1 Eyes Open")

---
## C2: Eyes Closed

Real EEG recording with eyes closed.

**Expected:** Enhanced alpha (7-13 Hz), especially in occipital channels

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# C2: EYES CLOSED
# ═══════════════════════════════════════════════════════════════════════════════

input("Subject: Keep eyes CLOSED. Press Enter to start recording...")
results_ec = capture_and_analyze("C2 Eyes Closed")

---
## Custom Test

Run any custom test by specifying a name.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CUSTOM TEST
# ═══════════════════════════════════════════════════════════════════════════════

TEST_NAME = "Custom Test"  # ← EDIT THIS
DURATION = 5.0             # ← EDIT THIS (seconds)

results_custom = capture_and_analyze(TEST_NAME, duration_s=DURATION)